# admin_routes

> Extended admin routes for dashboards and bulk operations

In [ ]:
#| default_exp admin_routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from dataclasses import fields
from ship_kit.permissions import require_role

## Admin Dashboard

A simple dashboard showing all managed models with counts and links.

In [ ]:
#| export
def admin_dashboard(rt, models, path='/admin'):
    """Simple dashboard showing all models.
    
    Args:
        rt: FastHTML route object
        models: List of tuples (name, table, model_class)
        path: Dashboard path (default: '/admin')
    """
    @rt(path)
    def get(req, sess):
        if not require_role('admin', req, sess):
            return RedirectResponse('/login', status_code=303)
        
        cards = []
        for name, table, _ in models:
            count = len(table())
            cards.append(
                Card(
                    H3(name.title()),
                    P(f"{count} items"),
                    A("Manage →", href=f"/admin/{name}/")
                )
            )
        
        return Container(
            H1("Admin Dashboard"),
            Grid(*cards, cols=3)
        )

## Detail Views

Read-only detail views for individual items.

In [ ]:
#| export
def admin_detail(rt, name, table, model_class):
    """Show single item details.
    
    Args:
        rt: FastHTML route object
        name: Model name (e.g., 'product')
        table: Database table
        model_class: Dataclass model
    """
    @rt(f"/admin/{name}/{{id}}")
    def get(req, sess, id: int):
        if not require_role('admin', req, sess):
            return RedirectResponse('/login', status_code=303)
        
        item = table[id]
        field_divs = []
        
        for field in fields(model_class):
            field_divs.append(
                Div(
                    Label(field.name.replace('_', ' ').title()),
                    P(str(getattr(item, field.name, '')))
                )
            )
        
        return Container(
            H1(f"{name.title()} Details"),
            Card(*field_divs),
            A("Edit", href=f"/admin/{name}/{item.id}/edit"),
            A("Back", href=f"/admin/{name}/")
        )

## Bulk Operations

Delete multiple items at once.

In [ ]:
#| export
def admin_bulk_delete(rt, name, table):
    """Delete multiple items at once.
    
    Args:
        rt: FastHTML route object
        name: Model name (e.g., 'product')
        table: Database table
    """
    @rt(f"/admin/{name}/delete", methods=['POST'])
    async def post(req, sess):
        if not require_role('admin', req, sess):
            return RedirectResponse('/login', status_code=303)
        
        form = await req.form()
        selected = form.getlist('selected')
        
        for item_id in selected:
            try:
                table.delete(int(item_id))
            except:
                pass  # Ignore errors for individual items
        
        # Redirect back to list view
        return RedirectResponse(f"/admin/{name}/", status_code=303)

## Usage Example

Here's how to use these admin route extensions with your existing admin setup:

In [ ]:
#| eval: false
from ship_kit.admin import setup_admin_routes
from ship_kit.admin_routes import admin_dashboard, admin_detail, admin_bulk_delete
from fasthtml.common import *
from dataclasses import dataclass
from typing import Optional

app, rt = fast_app()
db = Database('app.db')

# Define your models
@dataclass
class Product:
    name: str
    price: float
    id: Optional[int] = None

@dataclass
class User:
    username: str
    email: str
    id: Optional[int] = None

# Create tables
products = db.create(Product, pk='id')
users = db.create(User, pk='id')

# Your models list
models = [
    ('products', products, Product),
    ('users', users, User)
]

# 1. Basic CRUD (already exists)
for name, table, model in models:
    setup_admin_routes(rt, name, table, model)

# 2. Add dashboard
admin_dashboard(rt, models)

# 3. Add detail views (only if needed)
for name, table, model in models:
    admin_detail(rt, name, table, model)

# 4. Add bulk delete (only if needed)
for name, table, model in models:
    admin_bulk_delete(rt, name, table)

## Interactive Demo

Let's create a demo showing the dashboard in action:

In [ ]:
#| eval: false
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi
from monsterui.all import *
from dataclasses import dataclass
from typing import Optional

# Demo setup
demo_app, demo_rt = fast_app(hdrs=Theme.blue.headers())
demo_db = Database(":memory:")

@dataclass
class DemoProduct:
    name: str
    price: float
    id: Optional[int] = None

@dataclass
class DemoUser:
    username: str
    email: str
    id: Optional[int] = None

# Create tables
demo_products = demo_db.create(DemoProduct, pk='id')
demo_users = demo_db.create(DemoUser, pk='id')

# Add sample data
demo_products.insert(name="Widget", price=19.99)
demo_products.insert(name="Gadget", price=29.99)
demo_users.insert(username="alice", email="alice@example.com")
demo_users.insert(username="bob", email="bob@example.com")

# Mock auth for demo - override the imported require_role
import ship_kit.permissions
ship_kit.permissions.require_role = lambda role, req, sess: True

# Import admin functions
from ship_kit.admin import setup_admin_routes
from ship_kit.admin_routes import admin_dashboard, admin_detail

# Set up models
models = [
    ('products', demo_products, DemoProduct),
    ('users', demo_users, DemoUser)
]

# Add basic CRUD routes first
for name, table, model in models:
    setup_admin_routes(demo_rt, name, table, model)

# Add dashboard
admin_dashboard(demo_rt, models)

# Add detail views
for name, table, model in models:
    admin_detail(demo_rt, name, table, model)

# Add a home route that redirects to admin
@demo_rt('/')
def home():
    return RedirectResponse('/admin', status_code=303)

# Start the demo server
print("Starting admin demo server...")
print("\nDemo routes:")
print("- http://localhost:8000/admin - Admin Dashboard")
print("- http://localhost:8000/admin/products/ - Products List")
print("- http://localhost:8000/admin/users/ - Users List")

server = JupyUvi(demo_app)

In [ ]:
#| eval: false
# Remember to stop the server when done to avoid dangling threads
server.stop()